# **Colab to design an LC experiment with Trimmed Match**

In [0]:
#@title **Getting Started**

#@markdown Connect to the hosted runtime and run each cell after updating the necessary inputs


# Data input

In [0]:
#@title Load the libraries needed for the design     

BAZEL_VERSION = '3.0.0'
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!sudo apt-get install python3-dev python3-setuptools git
!git clone https://github.com/google/trimmed_match
!python3 -m pip install ./trimmed_match
!pip install colorama
!pip install gspread-dataframe

"""Loading the necessary python modules."""
import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import re
from scipy import stats

from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

import gspread
import warnings
from colorama import Fore, Style
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.colab import data_table
from google.colab import widgets
from oauth2client.client import GoogleCredentials
from google.colab import drive
from trimmed_match.design.common_classes import GeoXType, TimeWindow
from trimmed_match.design.trimmed_match_design import TrimmedMatchGeoXDesign
from trimmed_match.design.util import find_days_to_exclude, overlap_percent
from trimmed_match.design.util import check_time_periods
from trimmed_match.design.util import human_readable_number
from trimmed_match.design.util import expand_time_windows

warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"

In [0]:
#@markdown ---
#@markdown ### Enter the trix url for the sheet file containing the Client Sales Data:                               
#@markdown The spreadsheet should contain the mandatory columns:
#@markdown * date: date in the format YYYY-MM-DD
#@markdown * geo: the number which identifies the geo
#@markdown * response: variable on which you want to measure incrementality
#@markdown (e.g. sales, transactions)
#@markdown * cost: variable used as spend proxy (e.g. ad spend)

#@markdown Other columns can be present in the spreadsheet.

client_sales_table = "https://docs.google.com/spreadsheets/d/1MkkvWRsi9d4VvNwl908ylyYZW38B8DsBxTFbWE0Qs8M/edit#gid=0" #@param {type:"string"}

#@markdown ---

## load the trix in input

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
wks = gc.open_by_url(client_sales_table).sheet1
data = wks.get_all_values()
headers = data.pop(0)
geo_level_time_series = pd.DataFrame(data, columns=headers)

geo_level_time_series["date"] = pd.to_datetime(geo_level_time_series["date"])
for colname in ["response", "cost", "geo"]:
  geo_level_time_series[colname] = pd.to_numeric(geo_level_time_series[colname])

## set parameters in other cells of the colab based on the loaded data
number_of_weeks_test = 4 # length of the experiment in weeks
number_of_days_test = number_of_weeks_test * 7


# Select the parameters for the design and calculate the RMSE

In [0]:
#@title Select the parameters for the design of the experiment  

use_cross_validation = True

## set GeoX type, the type of the experiment: hold-back, heavy-up,
## heavy-down, control, go-dark, see common_classes.GeoXType for
## more details.
geox_type = GeoXType.HOLD_BACK


## Under the standard hypothesis testing framework, given a confidence_level
## (input) % confidence level for a one-sided
## test, the design has power_level (input) % power for testing
## H0: iROAS = 0 vs. H1: iROAS >= rmse_multiplier * rmse.
confidence_level = 0.90 #@param {type:"number"}
power_level = 0.80 #@param {type:"number"}
z_values = -1 * stats.norm.ppf(power_level)
rmse_multipliers = abs(stats.norm.ppf(1-confidence_level)) - z_values

design_start_date = "\"2020-01-01\"" #@param {type:"date"}
design_end_date = "\"2020-12-29\"" #@param {type:"date"}
eval_start_date = "\"2020-12-02\"" #@param {type:"date"}
aa_test_start_date = "\"2020-11-04\"" #@param {type:"date"}

experiment_duration_in_weeks = 4 #@param {type:"integer"}

#@markdown List the maximum budget for the experiment e.g. 300000
experiment_budget = "300000" #@param{type: "string"}
experiment_budget = float(experiment_budget)

#@markdown List any alternative budget which you would like to test separated
#@markdown by a comma, e.g. 125000, 150000
alternative_budget = "125000" #@param{type: "string"}
additional_budget = [float(re.sub(r"\W+", "", x)) for x in
                     alternative_budget.split(',') if alternative_budget != ""]

## The minimum detectable iROAS is defined as the value of the true iROAS such
## that, given a confidence_level (input) % confidence level for a one-sided
## test, gives a power_level (input) % power if the true iROAS is equal to the
## minimum detectable iROAS.
minimum_detectable_iROAS =  3#@param{type: "number"}
#@markdown Use an average order value of 1 if the design is based on sales/revenue or an actual average order value (e.g. 80$) for a design based on transactions/footfall/contracts.
average_order_value =  1#@param{type: "number"}

## Additional constraints which will be flagged in red if not met in
## the design

# upper bound on the minimal detectable relative lift
minimum_detectable_lift_in_response_metric = 0.1 * 100
# lower bound on the baseline revenue covered by the treatment group
minimum_revenue_covered_by_treatment = 0.05 * 100

#@markdown List the geo_id of the geos you want to exclude separated by
#@markdown a comma e.g. 100,200. Leave empty to select all geos.
geos_exclude = "2,13,14" #@param {type: "string"}
geos_exclude = [] if geos_exclude == "" else [re.sub(r"\W+", "", x) for x in
                                              geos_exclude.split(',')]

#@markdown List the days and time periods that you want to exclude separated by
#@markdown a comma e.g. 2019/10/10, 2010/10/11, 2018/10/20-2018/11/20. The format for time periods
#@markdown is "YYYY/MM/DD - YYYY/MM/DD", where the two dates specify the
#@markdown start and end date for the period. The format for
#@markdown day is "YYYY/MM/DD". Leave empty to use all days/weeks.
day_week_exclude = "2020/03/02-2020/03/22, 2020/08/10, 2020/08/14" #@param {type: "string"}
day_week_exclude = [] if day_week_exclude == "" else [
    re.sub(r"\s+", "", x) for x in day_week_exclude.split(",")
]


## convert input dates to datetimes (needed due to interactive parameters)
design_start_date = pd.to_datetime(design_start_date.replace("\"",""))
design_end_date = pd.to_datetime(design_end_date.replace("\"",""))
eval_start_date = pd.to_datetime(eval_start_date.replace("\"",""))
aa_test_start_date = pd.to_datetime(aa_test_start_date.replace("\"",""))

number_of_days_test = experiment_duration_in_weeks * 7
eval_end_date = eval_start_date + datetime.timedelta(days=number_of_days_test-1)
aa_test_end_date = aa_test_start_date + datetime.timedelta(
    days=number_of_days_test - 1)

design_start_date = min(design_start_date, aa_test_start_date, eval_start_date)
design_end_date = max(design_end_date, aa_test_end_date, eval_end_date)

## Find all the days we should exclude from the analysis from the input
periods_to_exclude = find_days_to_exclude(day_week_exclude)
days_exclude = expand_time_windows(periods_to_exclude)

## remove the excluded days from the rest of the analysis
geo_time_series = geo_level_time_series.copy()
geo_time_series = geo_time_series[~geo_time_series["date"].isin(days_exclude)]


## check that the user doesn't attempt by mistake to remove
## days/weeks in the evaluation or AA test periods.
days_in_eval = [
    x for x in geo_level_time_series["date"].drop_duplicates()
    if x in pd.Interval(eval_start_date, eval_end_date, closed="both")
]

days_in_aa_test = [x for x in geo_level_time_series["date"].drop_duplicates()
       if x in pd.Interval(aa_test_start_date, aa_test_end_date, closed="both")]

percentage_overlap_eval = overlap_percent(days_exclude, days_in_eval)
if percentage_overlap_eval > 0:
  raise ValueError((f'{Fore.RED}WARNING: {percentage_overlap_eval:.2} % of  ' +
                    f'the evaluation time period overlaps with days/weeks ' +
                    f'excluded in input. Please change eval_start_date.' +
                    f'\n{Style.RESET_ALL}'))

percentage_overlap_aa_test = overlap_percent(days_exclude, days_in_aa_test)
if percentage_overlap_aa_test > 0:
  raise ValueError(f'{Fore.RED}WARNING: {percentage_overlap_aa_test:.2} % of '
                   f'the aa test  time period overlaps with days/weeks ' +
                   f'excluded in input. Please change aa_test_start_date.' +
                   f'\n{Style.RESET_ALL}')


## check that the evaluation and AA test periods do not
## overlap (if the user has changed them)
percentage_overlap_eval_aa_test = overlap_percent(days_in_eval,
                                                  days_in_aa_test)
if percentage_overlap_eval_aa_test > 0:
  raise ValueError(f'{Fore.RED}WARNING: part of the evaluation time period ' +
                   f'overlaps with the AA test period. Please change ' +
                   f'eval_start_date.\n{Style.RESET_ALL}')

try:
  pass_checks = check_time_periods(geox_data=geo_level_time_series,
                       start_date_eval=eval_start_date,
                       start_date_aa_test=aa_test_start_date,
                       experiment_duration_weeks=experiment_duration_in_weeks,
                       frequency="infer")
except Exception as e:
  print(f'{Fore.RED} ERROR: ' + str(e) + f'\n{Style.RESET_ALL}')
  error_raised = e
  pass_checks = False

## check that the number of geos is even
geos_exclude = [int(x) for x in geos_exclude]
num_geos = len(set(geo_level_time_series["geo"].to_list()) - set(geos_exclude))
if num_geos % 2 != 0:
  geo_level_data = geo_level_time_series.groupby(
      "geo", as_index=False)["response"].sum()
  largest_geo = geo_level_data.loc[geo_level_data["response"].idxmax()]
  print(f'\nSince the number of geos is odd, we have removed the following' +
  f' geo (the one with largest response):')
  interactive_table.Create(largest_geo)
  geos_exclude.append(largest_geo["geo"])

In [0]:
#@title Summary of the possible designs 

if min([percentage_overlap_eval, percentage_overlap_aa_test,
        percentage_overlap_eval_aa_test]) > 0:
  raise ValueError(f'{Fore.RED}Either the evaluation time period or the AA ' +
                   f'test period overlaps with days/weeks excluded in input, ' +
                   f'or these two periods overlap. Please change them ' +
                   f'accordingly.\n{Style.RESET_ALL}')

if not pass_checks:
  raise ValueError(f'{Fore.RED} There is an error with the evaluation or' +
  f' aa test period in the previous colab cell.\nPlease correct that error' +
  f' and then rerun this cell.\n' +
  f'Previous error:\n' + str(error_raised) + '{Style.RESET_ALL}')

## set the number of excluded pairs to be tested
num_geos = len(set(geo_level_time_series["geo"].to_list()) - set(geos_exclude))
max_num_geopairs_trim = int(np.floor(num_geos/2 - 10))

candidate_num_pairs_excluded = range(0, max_num_geopairs_trim+1, 1)
## set the number of simulation used to compute the RMSE
number_of_simulations = 200

### Evaluate the RMSE of the possible designs ###

## remove the AA test period to make sure it's not used in the evaluation
## of the RMSE or in the training.
data_without_aa_test_period = geo_time_series[
    (geo_time_series["date"] < aa_test_start_date) |
    (geo_time_series["date"] > aa_test_end_date)]

data_for_design = data_without_aa_test_period.copy()
data_for_design = data_for_design[~data_for_design["geo"].isin(geos_exclude)]

time_window_for_design = TimeWindow(design_start_date, design_end_date)
time_window_for_eval = TimeWindow(eval_start_date, eval_end_date)
## initialize the TrimmedMatchGeoxDesign
pretest = TrimmedMatchGeoXDesign(
    geox_type=geox_type,
    pretest_data=data_for_design,
    response="response",
    spend_proxy="cost",
    matching_metrics={"response": 1.0, "cost": 0.01},
    time_window_for_design=time_window_for_design,
    time_window_for_eval=time_window_for_eval)

## run a first design with budget equal to the max. budget
preliminary_results, prel_results_detailed = pretest.report_candidate_designs(
    budget_list=[experiment_budget],
    iroas_list=[0],
    use_cross_validation=use_cross_validation,
    num_pairs_filtered_list=candidate_num_pairs_excluded,
    num_simulations=number_of_simulations)

## calculate the minimum detectable iROAS for a design with max. budget
chosen_design = preliminary_results.loc[
    preliminary_results["rmse_cost_adjusted"].idxmin()]
lowest_detectable_iroas = rmse_multipliers * chosen_design["rmse"]

## two cases are possible:
##   1) if the minimum detectable iROAS with max. budget is greater than
##      the minimum_detectable_iROAS in input, then calculate the budget needed
##      to reach the minimum_detectable_iROAS in input and run a design with
##      such budget. This is the code in the if clause below;
##   2) if the minimum detectable iROAS with max. budget is smaller than
##      the minimum_detectable_iROAS in input, then run designs with
##      budgets equal to the max. budget plus/minus 20%.
##      This is the code in the else clause below;
minimum_iroas_aov = minimum_detectable_iROAS / average_order_value
if lowest_detectable_iroas > minimum_iroas_aov:
  budget_to_reach_min_det_iroas = (experiment_budget * lowest_detectable_iroas
                                   / minimum_iroas_aov)
  additional_results, results_detailed = pretest.report_candidate_designs(
    budget_list=[budget_to_reach_min_det_iroas] + additional_budget,
    iroas_list=[0],
    spend_proxy="cost",
    use_cross_validation=use_cross_validation,
    num_pairs_filtered_list=candidate_num_pairs_excluded,
    num_simulations=number_of_simulations)

  results = pd.concat([preliminary_results, additional_results], sort=False)

else:
  optimal_budget = (experiment_budget * lowest_detectable_iroas /
                    minimum_iroas_aov)
  lower_budget = optimal_budget *  0.8
  upper_budget = optimal_budget * 1.2
  list_of_budgets = [lower_budget, optimal_budget, upper_budget
                    ] + additional_budget
  results, results_detailed = pretest.report_candidate_designs(
      budget_list=list_of_budgets,
      iroas_list=[0],
      use_cross_validation=use_cross_validation,
      num_pairs_filtered_list=candidate_num_pairs_excluded,
      num_simulations=number_of_simulations)


filtered_pairs = preliminary_results.loc[
    preliminary_results["rmse_cost_adjusted"].idxmin(), "num_pairs_filtered"]
axes_paired = pretest.output_chosen_design(filtered_pairs, base_seed=0,
                                           confidence=confidence_level)
plt.close()

## assign geos to treatment and control groups
geopairs = pretest._geo_level_eval_data
geopairs.sort_values(by=["pair"], inplace=True)

geo_treatment = geopairs[geopairs["assignment"]==1]
geo_control = geopairs[geopairs["assignment"]==0]
treatment_geo = geo_treatment["geo"].to_list()
control_geo = geo_control["geo"].to_list()

budgets_for_design = results["budget"].drop_duplicates().to_list()

### AA test to check the coverage probability of the confidence interval ###

## remove the evaluation period to make sure it's not used in the evaluation
## of the RMSE or in the training.
data_without_eval_period = geo_time_series[
    (geo_time_series["date"] < eval_start_date) |
    (geo_time_series["date"] > eval_end_date)]


data_for_aa_test = data_without_eval_period.copy()
data_for_aa_test = data_for_aa_test[~data_for_aa_test["geo"].isin(geos_exclude)]

time_window_for_aa_test = TimeWindow(aa_test_start_date,
                                                    aa_test_end_date)
## initialize the TrimmedMatchGeoxDesign
aa_test_class = TrimmedMatchGeoXDesign(
    geox_type=geox_type,
    pretest_data=data_for_aa_test,
    response="response",
    spend_proxy="cost",
    matching_metrics={"response": 1.0, "cost": 0.01},
    time_window_for_design=time_window_for_design,
    time_window_for_eval=time_window_for_aa_test)

## calculate the point estimate for each simulation
aa_results, aa_results_detailed = aa_test_class.report_candidate_designs(
    budget_list=[budgets_for_design[0]],
    iroas_list=[0],
    use_cross_validation=use_cross_validation,
    num_pairs_filtered_list=candidate_num_pairs_excluded,
    num_simulations=number_of_simulations)


## The code below this line only takes care of formatting the output

## Create a "designs" table with the attempted designs and their info
designs = []
for budget in budgets_for_design:
  tmp_result = results[results["budget"]==budget]
  chosen_design = tmp_result.loc[tmp_result["rmse_cost_adjusted"].idxmin()]
  baseline = geo_treatment.loc[
      geo_treatment["pair"] > chosen_design["num_pairs_filtered"],
      "response"].sum()

  min_detectable_iroas = (
      average_order_value * rmse_multipliers * chosen_design["rmse"])
  min_detectable_lift = (
      budget * 100 * rmse_multipliers * chosen_design["rmse"] / baseline)
  num_removed_geos = int(2 * chosen_design["num_pairs_filtered"])
  num_geo_pairs = int((num_geos - num_removed_geos) / 2)
  treat_control_removed = (f'{num_geo_pairs}  /  {num_geo_pairs}  / ' +
  f'{num_removed_geos}')
  revenue_covered = 100 * baseline / geopairs["response"].sum()
  national_budget = human_readable_number(
      budget / chosen_design["proportion_cost_in_experiment"])
  designs.append({
      "Budget": human_readable_number(budget),
      "Minimum detectable iROAS": f'{min_detectable_iroas:.3}',
      "Minimum detectable lift in response": f'{min_detectable_lift:.2f} %',
      "Treatment/control/excluded geos": treat_control_removed,
      "Revenue covered by treatment group": f'{revenue_covered:.2f} %',
      "Cost/baseline response": f'{(budget / baseline * 100):.2f} %',
      "Cost if test budget is scaled nationally": national_budget
  })



## define function to colorcode rows and cells of the output table
def is_optimal_design(row):
    """Color a row in:
    - red if the minimum detectable iROAS of the corresponding design
      is larger than the target
    - green if its equal to the target
    - beige if it's lower than the target
    """
    if float(row["Minimum detectable iROAS"]) == minimum_detectable_iROAS:
          return pd.Series('background-color: lightgreen', row.index)
    elif float(row["Minimum detectable iROAS"]) > minimum_detectable_iROAS:
          return pd.Series('background-color: tomato', row.index)
    else:
          return pd.Series('background-color: beige', row.index)

def flag_warning_lift(val, value):
    """
    Color a cell in red if its value is larger than the value
    in input
    """
    color = 'red' if float(val.strip(' %')) > value else 'black'
    return 'color: %s' % color

def flag_warning_revenue(val, value):
    """
    Color a cell in red if its value is smaller than the value
    in input
    """
    color = 'red' if float(val.strip(' %')) < value else 'black'
    return 'color: %s' % color


## convert the table to a pd.DataFrame and select a subset of columns
designs = pd.DataFrame(designs)
designs.index.rename("Design", inplace=True)
designs = designs[["Budget", "Minimum detectable iROAS",
                   "Minimum detectable lift in response",
                   "Treatment/control/excluded geos",
                   "Revenue covered by treatment group",
                   "Cost/baseline response",
                   "Cost if test budget is scaled nationally"]]

## apply colorcoding to rows and cells of the table
designs_table = designs.style.applymap(
    flag_warning_lift,
    value=minimum_detectable_lift_in_response_metric,
    subset=["Minimum detectable lift in response"]).applymap(
        flag_warning_revenue,
        value=minimum_revenue_covered_by_treatment,
        subset=["Revenue covered by treatment group"]).apply(
            is_optimal_design, axis=1)

designs_table
recommended_design = designs.index[pd.to_numeric(
    designs['Minimum detectable iROAS']) == minimum_detectable_iROAS][0]


In [0]:
#@title Select the design to be used in the experiment  
#@markdown Select the design using the number as displayed in the table in
#@markdown the cell called "Summary of the possible designs".

selected_design =   1#@param {type:"integer"}

if selected_design not in designs.index:
  raise ValueError(f'the selected design must be one of {designs.index.to_list()}, got {selected_design}')

selected_design = int(selected_design)
final_design = designs[designs.index == selected_design]
selected_budget = final_design["Budget"].values[0]
filtered = final_design["Treatment/control/excluded geos"].values[0]
filtered_pairs = int(filtered.replace(" ", "").split('/')[2]) // 2


## Uncomment the following line to override the automatic choice for
## the number of filtered pairs. For example, using filtered_pairs = 5 will
## filter out the pairs 1, 2, 3, 4, 5.

# filtered_pairs = 5

###


axes_paired = pretest.output_chosen_design(filtered_pairs, base_seed=0,
                                           confidence=confidence_level)
plt.close()

In [0]:
#@title Scatterplot and time series comparison of different metrics for treatment vs. control groups  

for ax in axes_paired[1]:
  ylim = ax.get_ylim()
  for period in periods_to_exclude:
    if period.first_day < period.last_day:
      useless=ax.fill_between([period.first_day, period.last_day], ylim[0], ylim[1],
                              facecolor="gray", alpha=0.5)
    else:
      useless=ax.vlines(period.first_day, ylim[0], ylim[1],
                         color="gray", alpha=0.5)

  handles, labels = ax.get_legend_handles_labels()
  patch = mpatches.Patch(color='grey', label='removed by the user')
  handles.append(patch)
  useless=ax.fill_between([aa_test_start_date, aa_test_end_date],  ylim[0],
                          ylim[1], facecolor="g", alpha=0.5)
  patch = mpatches.Patch(color='g', label='left out for AA test')
  handles.append(patch)
  useless=ax.legend(handles=handles, loc='best')

axes_paired[1,1].figure

In [0]:
#@title Plot each pair of geos for comparison  
g = pretest.plot_pair_by_pair_comparison()

# Summary of the design and save the pretest data, the geopairs, treatment and control stores in a trix.

In [0]:
#@title Summary and Results  

included_pairs = geopairs["pair"] > filtered_pairs
treatment_geo = geopairs.loc[(included_pairs &
                              (geopairs["assignment"] == 1))]["geo"].to_list()
control_geo = geopairs.loc[(included_pairs &
                            (geopairs["assignment"] == 0))]["geo"].to_list()

temporary = geo_level_time_series[geo_level_time_series["geo"].isin(
    treatment_geo)]
treatment_time_series = temporary[
    (temporary["date"] >= design_start_date)
    & (temporary["date"] <= design_end_date)].groupby(
        "date", as_index=False)["response", "cost"].sum()

temporary = geo_level_time_series[geo_level_time_series["geo"].isin(
    control_geo)]
control_time_series = temporary[
    (temporary["date"] >= design_start_date)
    & (temporary["date"] <= design_end_date)].groupby(
        "date", as_index=False)["response", "cost"].sum()

eval_window = ((treatment_time_series['date'] >= eval_start_date) &
                (treatment_time_series['date'] <= eval_end_date))
baseline = treatment_time_series[eval_window]["response"].sum()

result_to_out = results[results["budget"] ==
                        budgets_for_design[selected_design]]

print("Data in input:\n")
print("-  {} Geos \n".format(
    len(geo_level_time_series["geo"].drop_duplicates().index)))

print("Output:\n")
print("The output contains two lists of geos: one for treatment" +
      " and the other for control\n")

human_baseline = human_readable_number(baseline)
cost_baseline = budgets_for_design[selected_design] * 100 / baseline
print("-  {} Geo pairs for the experiment\n".format(len(treatment_geo)))
print("    Baseline store response: ${} for treatment\n".format(human_baseline))
print("    Cost/baseline = ${} / ${} ~ {:.3}%\n".format(selected_budget,
                                                        human_baseline,
                                                        cost_baseline))

print(f'Minimum detectable iROAS = ' +
      f'{final_design["Minimum detectable iROAS"].values[0]}')
print(f'Minimum detectable lift in % = ' +
      f'{final_design["Minimum detectable lift in response"].values[0]}')

print(f"The design has Power {100 * power_level:.3}+% with Type-I error " +
      f"{100 *(1 - confidence_level):.3}% for testing H0: iROAS=0 vs " +
      f"H1: iROAS >= {final_design['Minimum detectable iROAS'].values[0]}")

In [0]:
#@title Report stores for treatment and control separately and write to trix 

#@markdown ###Insert the name google sheets in which we will save the data.
#@markdown The trix contains 4 worksheets, named:
#@markdown * "pretest data", containing the geo level time series;
#@markdown * "geopairs", containing the pairs of geos and their assignment.
#@markdown * "treatment geos", contains the list of geos in the treatment;
#@markdown * "control geos", contains the geos in the control groups.
Client_Name = "Client_Name" #@param {type:"string"}
filename_design = Client_Name + "_design.csv" #@param {type:"string"}

geopairs_formatted = geopairs.copy()
geopairs_formatted.loc[geopairs_formatted["pair"] <= filtered_pairs,
                       "assignment"] = -1
geopairs_formatted["assignment"] = geopairs_formatted["assignment"].map({
    -1: "Filtered",
    0: "Control",
    1: "Treatment"
})

geo_level_time_series["period"] = [
   # -3 indicates days excluded
   -3 if x in days_exclude else (
   # 0 indicates days in the evaluation period
   0 if x>=eval_start_date and x<=eval_end_date else (
   # -2 indicates days in the aa_test period
   -2 if x >= aa_test_start_date and x <= aa_test_end_date
   # -1 indicates days in the training period
   else -1))
   for x in geo_level_time_series["date"]
]

tmp = geo_level_time_series[geo_level_time_series["geo"].isin(treatment_geo +
                                                              control_geo)]
design_data = tmp.merge(
    geopairs[["geo", "pair", "assignment"]], on="geo", how="left")


sh = gc.create(filename_design)
wid = sh.add_worksheet("pretest data", rows=1, cols=1)
set_with_dataframe(wid, design_data)
wid = sh.add_worksheet("geopairs", rows=1, cols=1)
set_with_dataframe(wid, geopairs_formatted)
wid = sh.add_worksheet("treatment geos", rows=1, cols=1)
set_with_dataframe(wid, pd.DataFrame({"geo": treatment_geo}))
wid = sh.add_worksheet("control geos", rows=1, cols=1)
set_with_dataframe(wid, pd.DataFrame({"geo": control_geo}))
out = sh.del_worksheet(sh.sheet1)

# Appendix:

In [0]:
#@title Plot of the RMSE as a function of the # of trimmed pairs  

#@markdown The first graph below shows the RMSE of the iROAS estimator
#@markdown with respect to the number of excluded geo pairs,
#@markdown with the baseline store sales (treatment group + control group)
#@markdown displayed next to each point of (# excluded pairs, RMSE).

#@markdown The second graph below shows the proportion of confidence
#@markdown intervals that cointains zero in an A/A test.

aa_test_result = []
for num_pairs_filtered in candidate_num_pairs_excluded:
  temp_df = aa_results_detailed[(budgets_for_design[0], 0, num_pairs_filtered)]
  temp_df["contains_zero"] = (temp_df["conf_interval_low"] <= 0) & (
    temp_df["conf_interval_up"] >= 0)
  coverage = temp_df['contains_zero'].mean()
  aa_test_result.append({"coverage": coverage,
                         "num_pairs_filtered": num_pairs_filtered,
                         "budget": budgets_for_design[0],
                         "ci_level": temp_df["ci_level"][0]})

aa_test_result = pd.DataFrame(aa_test_result)

# call the function that creates all the axis
axes_dict = pretest.plot_candidate_design(results=results)
## display the results of the designs with different budgets in different tabs
list_of_str = ["budget = " + human_readable_number(budget)+"$"
               for budget in budgets_for_design]
tb = widgets.TabBar(list_of_str)
for ind in range(len(budgets_for_design)):

  result = results[results["budget"]==budgets_for_design[ind]].reset_index(
      drop=True)
  with tb.output_to(ind):
    ax = axes_dict[(budgets_for_design[ind], 0)].axes[0]
    labels = ax.get_yticks().tolist()
    labels = [str(round(float(x)*average_order_value, 2)) for x in labels]
    useless = ax.set_yticklabels(labels)
    display(axes_dict[(budgets_for_design[ind], 0)])

    print_result = result.copy()
    print_result.rename(columns={"num_pairs_filtered": "# of excluded pairs"},
                        inplace=True)
    print_result["rmse"] = print_result["rmse"] * average_order_value
    print_result["rmse_cost_adjusted"] = (print_result["rmse_cost_adjusted"]
                                          * average_order_value)
    data_table.DataTable(print_result[["# of excluded pairs", "rmse",
                 "rmse_cost_adjusted", "experiment_response"]],
                 include_index=False)

fig1 = plt.figure(figsize=(15, 7.5))
ax1 = fig1.add_subplot(1, 1, 1)
useless=ax1.plot(
    aa_test_result['num_pairs_filtered'], aa_test_result['coverage'], 'blue',
    label='Coverage')
useless=ax1.hlines(
    y=aa_test_result['ci_level'][0],
    xmin=min(aa_test_result["num_pairs_filtered"]),
    xmax=max(aa_test_result["num_pairs_filtered"]),
    colors='red',
    linestyles='dashed',
    label='Confidence level (nominal)')
useless=ax1.set_xlabel('#(Excluded geo pairs)')
useless=ax1.set_ylabel('Coverage')
useless=ax1.set_title('A/A test confidence interval coverage')
useless=ax1.legend()